# Desafio 1: Base de dados CEAPS

<p style='text-align: justify'><font size=2.>CEAPS (cotas para exercício da atividade parlamentar dos senadores) é o valor que os membros do Poder Legislativo (deputados federais e estaduais, senadores e vereadores) têm direito a receber para cobrir despesas em função da sua atividade como parlamentar. Seu valor varia de acordo com o Estado do congressista com base na distância para Brasília. Ela é dividida em 7 cifras, que vão desde o aluguel de imóveis para escritório político a gastos com alimentação, combustível, passagens, hospedagem e divulgação da atividade parlamentar.</font></p>

### Dicionário dos dados

| Atributo | Significado |
| --- | --- |
| Ano | referente ao ano em que foi solicitado o reembolso do gasto |
| Mês | referente ao mês em que foi solicitado o reembolso do gasto |
| Senador| nome do Senador |
| tipo_despesa | tipo de gasto efetuado |
| CNPJ_CPF | CNPJ ou CPF da empresa/prestador do serviço |
| fornecedor | nome da empresa/prestador do serviço |
| documento | número do documento |
| data | data da despesa |
| detalhamento | detalhamento do gasto |
| valor_reembolsado | valor reembolsado ao Senador |
| cod_documento | código do documento |

### Bibiotecas básicas e outros importes

In [1]:
import glob
import chardet
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['font.family'] = 'Arial, Helvetica, sans-serif'

warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 200)
np.set_printoptions(suppress=True, precision=3)

### Concatenação das bases de 2018 a 2022

> Verifica a codificação da planilha e em seguinda realiza a concatenação

In [2]:
path = r'data/'
all_files = glob.glob(path + '/*.csv')
li = []

for filename in all_files:    
    with open(filename, 'rb') as f:
        result = chardet.detect(f.read())        
    df = pd.read_csv(filename, low_memory=False, sep=';', header=1, encoding=result['encoding'])
    li.append(df)
frame = pd.concat(li, axis=0, ignore_index=True)
frame.to_csv('data/despesa_ceaps_2018_2022.csv', index=False)

### Carregamento dos dados

In [3]:
despesas = pd.read_csv('data/despesa_ceaps_2018_2022.csv')
print(f'Linhas: {despesas.shape[0]} | Colunas: {despesas.shape[1]}')
despesas.head(2)

Linhas: 81179 | Colunas: 11


,ANO,MES,SENADOR,TIPO_DESPESA,CNPJ_CPF,FORNECEDOR,DOCUMENTO,DATA,DETALHAMENTO,VALOR_REEMBOLSADO,COD_DOCUMENTO
0,2020,1,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",05.914.650/0001-66,ENERGISA,14764742,06/01/2020,Despesa com energia elétrica para uso do escri...,"122,62",2141820
1,2020,1,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",05.914.650/0001-66,ENERGISA,14764741,06/01/2020,Despesa com energia elétrica para uso do escri...,"137,75",2141818


### Informações dos dados

In [4]:
despesas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81179 entries, 0 to 81178
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   ANO                81179 non-null  int64 
 1   MES                81179 non-null  int64 
 2   SENADOR            81179 non-null  object
 3   TIPO_DESPESA       81179 non-null  object
 4   CNPJ_CPF           81179 non-null  object
 5   FORNECEDOR         81179 non-null  object
 6   DOCUMENTO          77925 non-null  object
 7   DATA               81179 non-null  object
 8   DETALHAMENTO       50509 non-null  object
 9   VALOR_REEMBOLSADO  81179 non-null  object
 10  COD_DOCUMENTO      81179 non-null  int64 
dtypes: int64(3), object(8)
memory usage: 6.8+ MB


### Seleção de recursos

> Remove a coluna documento porque já temos a coluna com o código do documento que não possui valores nulos.

In [5]:
despesas.drop(['DOCUMENTO'], axis=1, inplace=True)

### Remoção registros nulos

In [6]:
print('Porcentagem de dados ausentes antes da remoção: ', despesas.isnull().sum().sum())
despesas.dropna(inplace=True)
print(f'Linhas: {despesas.shape[0]} | Coluna:  {despesas.shape[1]}')
print('Porcentagem de dados ausentes depois da remoção: ', despesas.isnull().sum().sum())
despesas.reset_index(drop=True, inplace=True)

Porcentagem de dados ausentes antes da remoção:  30670
Linhas: 50509 | Coluna:  10
Porcentagem de dados ausentes depois da remoção:  0


### Conversão coluna data

In [7]:
despesas['DATA'] = pd.to_datetime(despesas['DATA'], errors='coerce')
type(despesas['DATA'][0])

pandas._libs.tslibs.timestamps.Timestamp